<article class="day-desc"><h2>--- Day 5: Hydrothermal Venture ---</h2><p>You come across a field of <a href="https://en.wikipedia.org/wiki/Hydrothermal_vent" target="_blank">hydrothermal vents</a> on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.</p>
<p>They tend to form in <em>lines</em>; the submarine helpfully produces a list of nearby <span title="Maybe they're Bresenham vents.">lines of vents</span> (your puzzle input) for you to review. For example:</p>
<pre><code>0,9 -&gt; 5,9
8,0 -&gt; 0,8
9,4 -&gt; 3,4
2,2 -&gt; 2,1
7,0 -&gt; 7,4
6,4 -&gt; 2,0
0,9 -&gt; 2,9
3,4 -&gt; 1,4
0,0 -&gt; 8,8
5,5 -&gt; 8,2
</code></pre>
<p>Each line of vents is given as a line segment in the format <code>x1,y1 -&gt; x2,y2</code> where <code>x1</code>,<code>y1</code> are the coordinates of one end the line segment and <code>x2</code>,<code>y2</code> are the coordinates of the other end. These line segments include the points at both ends. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 1,3</code> covers points <code>1,1</code>, <code>1,2</code>, and <code>1,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,7</code> covers points <code>9,7</code>, <code>8,7</code>, and <code>7,7</code>.</li>
</ul>
<p>For now, <em>only consider horizontal and vertical lines</em>: lines where either <code>x1 = x2</code> or <code>y1 = y2</code>.</p>
<p>So, the horizontal and vertical lines from the above list would produce the following diagram:</p>
<pre><code>.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
</code></pre>
<p>In this diagram, the top left corner is <code>0,0</code> and the bottom right corner is <code>9,9</code>. Each position is shown as <em>the number of lines which cover that point</em> or <code>.</code> if no line covers that point. The top-left pair of <code>1</code>s, for example, comes from <code>2,2 -&gt; 2,1</code>; the very bottom row is formed by the overlapping lines <code>0,9 -&gt; 5,9</code> and <code>0,9 -&gt; 2,9</code>.</p>
<p>To avoid the most dangerous areas, you need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is anywhere in the diagram with a <code>2</code> or larger - a total of <code><em>5</em></code> points.</p>
<p>Consider only horizontal and vertical lines. <em>At how many points do at least two lines overlap?</em></p>
</article>

In [25]:
import Pkg; Pkg.add("Combinatorics"); using Combinatorics;
import Pkg; Pkg.add("Interpolations"); using Interpolations;
using Base.Iterators;

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [56]:
function input_line_to_points(input_line)
    pairs = split(input_line, " -> ");
    x1, y1 = [parse(Int64, n) for n in split(pairs[1], ",")];
    x2, y2 = [parse(Int64, n) for n in split(pairs[2], ",")];
    return ((x1,y1),(x2,y2))
end;

function init()
    input_lines = readlines("input.txt");
    points = [input_line_to_points(input_line) for input_line in input_lines];
    return points;
end;

function in_bounds(x, x1, x2)
    return ((x >= x1) && (x <= x2)) || ((x >= x2) && (x <= x1))
end;

function is_horizontal(((x1,y1), (x2,y2)))
    return y1 == y2
end;

function is_vertical(((x1,y1), (x2,y2)))
    return x1 == x2
end;

function as_range_pair(((x1,y1), (x2,y2)))
    return (x1:(x2 > x1 ? 1 : -1):x2, y1:(y2 > y1 ? 1 : -1):y2);
end;

In [62]:
lines = init();
points = Set();

v_or_h = filter((line) -> is_vertical(line) || is_horizontal(line), lines);
v_or_h_ranges = as_range_pair.(v_or_h);

for (l1, l2) in combinations(v_or_h_ranges, 2)
    x_range, y_range = (intersect(l1[1], l2[1]), intersect(l1[2], l2[2]));
    for point in product(x_range, y_range)
        push!(points, point);
    end;
end;
length(points)

6189

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider <em>diagonal lines</em>.</p>
<p>Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 3,3</code> covers points <code>1,1</code>, <code>2,2</code>, and <code>3,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,9</code> covers points <code>9,7</code>, <code>8,8</code>, and <code>7,9</code>.</li>
</ul>
<p>Considering all lines from the above example would now produce the following diagram:</p>
<pre><code>1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
</code></pre>
<p>You still need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is still anywhere in the diagram with a <code>2</code> or larger - now a total of <code><em>12</em></code> points.</p>
<p>Consider all of the lines. <em>At how many points do at least two lines overlap?</em></p>
</article>

In [61]:
lines = init();
points = Set();

ranges = as_range_pair.(lines);

function to_points(l)
    if length(l[1]) == 1
        return zip(cycle(l[1]), l[2]);
    elseif length(l[2]) == 1
        return zip(l[1], cycle(l[2]));
    else
        return zip(l[1], l[2]);
    end;
end

line_points = to_points.(ranges);

for (l1, l2) in combinations(line_points, 2)
    for point in intersect(l1, l2)
        push!(points, point);
    end;
end;

length(points)

19164

The above solution is widely considered to be gross.